In [1]:
from datasets import load_dataset

dataset = load_dataset("clue", "chid")

Dataset clue downloaded and prepared to /home/mymusise/.cache/huggingface/datasets/clue/chid/1.0.0/e508b66266ba417d60e89ed8b167699cb4b56d3a2ead29b5667907d08069dbfc. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 821.55it/s]


In [10]:
from tqdm import tqdm
from collections import defaultdict


dataset['train'][0].keys()
all_candidates = set()
counter_cand_len = defaultdict(int)
for example in dataset['train']:
    for can in example['candidates']:
        all_candidates.add(can)
    counter_cand_len[len(example['candidates'])] += 1
# dataset['train']['candidates']
print(len(all_candidates))
counter_cand_len

candidates_ids = {candi: i for i, candi in enumerate(all_candidates)}

3848


defaultdict(int, {10: 84709})

In [35]:
from transformers import AutoTokenizer


model_path = "/data2/CPM-distill/CPM-Generate-distill"

tokenizer = AutoTokenizer.from_pretrained(model_path)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CpmTokenizer'. 
The class this function is called from is 'XLNetTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [57]:
import re

replace = re.compile('#idiom\d{6}#')

def preprocess(example):
    features = []
    sentences = []
    labels = []
    for ans_i, (candidate_id, ans) in enumerate(zip(example['answers']['candidate_id'], example['answers']['text'])):
        for con_i, content in enumerate(example['content']):
            sentence = replace.sub(ans, content)
            if ans_i == con_i:
                label = 1
            else:
                label = 0
            # yield {'text': sentence, 'label': label}
            sentences.append(sentence)
            labels.append(label)
            # features.append({'text': sentence, 'label': label})
    features = tokenizer(sentences, max_length=256, truncation=True, padding=True, add_special_tokens=False, return_tensors='pt')
    features['label'] = labels
    return features
    # return features


# preprocess(dataset['train'][0])
train_dataset = dataset['train'].map(preprocess)
eval_dataset = dataset['validation'].map(preprocess)

  6%|▋         | 5321/84709 [02:18<31:50, 41.56ex/s]  

In [ ]:
from transformers import AutoModelForSequenceClassification

labels = [0, 1]

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=len(labels)+1)

In [ ]:
# ensure some model padding token id
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
import numpy as np
import evaluate


metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
print(f"before train: {trainer.evaluate()}")
trainer.train()
print(f"after train: {trainer.evaluate()}")